In [2]:
%load_ext autoreload
%autoreload 2

# AutoModel

In [8]:
from glasses.nn.models.AutoModel import AutoModel
from glasses.nn.models.AutoConfig import AutoConfig
import torch

In [9]:
AutoModel.from_name('resnet18')

ResNet(
  (encoder): ResNetEncoder(
    (stem): ResNetStem(
      (0): ConvBnAct(
        (conv): Conv2dPad(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): ReLU(inplace=True)
      )
      (1): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (layers): ModuleList(
      (0): Sequential(
        (0): ResNetLayer(
          (block): Sequential(
            (0): ResNetBasicBlock(
              (block): Sequential(
                (conv1): Conv2dPad(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                (act1): ReLU(inplace=True)
                (conv2): Conv2dPad(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                (bn2): BatchNorm2d(64, eps=1e-05, momentum=0

In [14]:
model = AutoModel.from_name('resnet18')
# notify to the model we want to have the features
model.encoder.features
model(torch.rand((1,3,224,224)))
# get the features :)
features = model.encoder.features
[print(f.shape) for f in features]

torch.Size([1, 64, 112, 112])
torch.Size([1, 64, 56, 56])
torch.Size([1, 128, 28, 28])
torch.Size([1, 256, 14, 14])


[None, None, None, None]

In [11]:
AutoConfig.from_name('unet')

Config(input_size=384, resize=384, mean=None, std=None, interpolation='bilinear')

# UNet

In [19]:
import torch
from glasses.nn.models.classification import *
from glasses.utils.Storage import ForwardModuleStorage
from glasses.nn.models.segmentation.unet import *
from glasses.nn.models.classification.resnet import ResNetEncoder, ResNet, ResNetBottleneckBlock
from glasses.nn.models.classification.efficientnet import *
from glasses.nn.models.classification import *
from torch import nn

In [31]:
unet = UNet.from_encoder(lambda *args, **kwargs: ResNet.resnet18(*args, **kwargs))

unet

UNet(
  (encoder): ResNetEncoder(
    (stem): ResNetStem(
      (0): ConvBnAct(
        (conv): Conv2dPad(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): ReLU6()
      )
      (1): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (layers): ModuleList(
      (0): Sequential(
        (0): ResNetLayer(
          (block): Sequential(
            (0): ResNetBasicBlock(
              (block): Sequential(
                (conv1): Conv2dPad(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                (act1): ReLU6()
                (conv2): Conv2dPad(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_r

In [33]:

unet = UNet.from_encoder(partial(AutoModel.from_name, 'resnet18'))
model.summary(input_shape=(1,384,384))

() {'in_channels': 1}
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         Conv2dPad-1         [-1, 64, 192, 192]           3,136
       BatchNorm2d-2         [-1, 64, 192, 192]             128
              ReLU-3         [-1, 64, 192, 192]               0
         MaxPool2d-4           [-1, 64, 96, 96]               0
         Conv2dPad-5           [-1, 64, 96, 96]          36,864
       BatchNorm2d-6           [-1, 64, 96, 96]             128
              ReLU-7           [-1, 64, 96, 96]               0
         Conv2dPad-8           [-1, 64, 96, 96]          36,864
       BatchNorm2d-9           [-1, 64, 96, 96]             128
         Identity-10           [-1, 64, 96, 96]               0
             ReLU-11           [-1, 64, 96, 96]               0
 ResNetBasicBlock-12           [-1, 64, 96, 96]               0
        Conv2dPad-13           [-1, 64, 96, 96]          36,864
      BatchNorm2d

(tensor(14228322), tensor(14228322), tensor(54.2767), tensor(410.3392))

# Benchmark

In [7]:
import timm
from glasses.nn.models import *
timm.create_model('seresnet50')

SENet(
  (layer0): Sequential(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu1): ReLU(inplace=True)
    (pool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  )
  (layer1): Sequential(
    (0): SEResNetBottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (se_module): SEModule(
        (avg_pool): Ada

In [8]:
SEResNet.cse_resnet50()

ResNet(
  (encoder): ResNetEncoder(
    (stem): ResNetStem(
      (0): ConvBnAct(
        (conv): Conv2dPad(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): ReLU(inplace=True)
      )
      (1): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (layers): ModuleList(
      (0): Sequential(
        (0): ResNetLayer(
          (block): Sequential(
            (0): ResNetBottleneckBlock(
              (block): Sequential(
                (conv1): Conv2dPad(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
                (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                (act1): ReLU(inplace=True)
                (conv2): Conv2dPad(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=

In [9]:
enc = EfficientNet.efficientnet_b0().encoder
x = torch.randn((1,3,224,224))

In [10]:
from glasses.nn.models.classification.efficientnet import *
import timm

from transfer_weights import clone_model
from benchmark import benchmark

src = timm.create_model('tf_efficientnet_lite1', pretrained='True')
dst = EfficientNetLite.efficientnet_lite1(mode='same')

In [11]:
def from_encoder(model_def, *args, **kwargs):
    model = model_def( *args, **kwargs)
    encoder = nn.Identity()
    if isinstance(model, ResNet):
        encoder = WithFeatures(model.encoder, 
                              stages = [
                                  model.encoder.stem[-2], 
                                  *model.encoder.layers,
                              ],
                              features_widths = model.encoder.widths[1:]
                             )
        
    elif isinstance(model, EfficientNet):
        encoder = WithFeatures(model.encoder, 
                              stages = [
                                  model.encoder.stem[-2], 
                                  model.encoder.layers[1],
                                  model.encoder.layers[2],
                                  model.encoder.layers[3],
                              ],
                              features_widths = [model.encoder.widths[0],
                                                 model.encoder.widths[2],
                                                 model.encoder.widths[3],
                                                 model.encoder.widths[4],
                                                ]
                             )
        
    return encoder

In [12]:
enc = ResNet.resnet18().encoder

In [13]:
enc.features_widths()

TypeError: 'list' object is not callable

In [ ]:
enc.stages()

In [ ]:
enc.features

enc(x)

enc.features


In [ ]:
m

In [ ]:
backbone = EfficientNet.efficientnet_b0().encoder


encoder = WithFeatures(backbone, 
                              stages = [
                                  backbone.stem[-2], 
                                  backbone.layers[1],
                                  backbone.layers[2],
                                  backbone.layers[3],
                              ],
                              features_widths = [32, 24, 40, 80]
                             )


m = UNet(encoder = lambda *args, **kwargs: encoder, decoder = partial(UNetDecoder, 
                                                                      widths=[1280, 256, 128, 64, 32, 16]))

In [ ]:
enc.features

In [ ]:
name = 'resnet-asdsa'

name.startswith('resnet')

In [ ]:
m.summary((3, 224, 224))

In [ ]:
dst = clone_model(src, dst)

In [ ]:
dst.summary()

In [ ]:
dst

In [ ]:
# import torch

# transform = dst.configs['efficientnet_lite1'].transform

# benchmark(dst.cuda(), transform)

In [ ]:
# benchmark(src.cuda(), transform)

In [ ]:
src = timm.create_model('efficientnet_b0', pretrained='True')
src

In [ ]:
dst.summary()